# CSE 202:  *Power Grid* Experiments
## Winter 2025
### Melina Dimitropoulou Kapsogeorgou, Andrew Pan, Adrian Rosing, Andrew Russell, Benjamin Xia

In [43]:
import itertools
import networkx as nx
from networkx.algorithms import tree

## Pre-Game Initialization

In [67]:
def phase0_solve_kmst(G, K):
    V = list(G.nodes)
    E = list(G.edges)

    result_mst = None
    min_weight = float('inf')
    for Vp in itertools.combinations(V, K):
        subgraph = G.subgraph(Vp)
        if not nx.is_connected(subgraph):
            continue
        T = nx.minimum_spanning_tree(subgraph, algorithm='prim')
        total_cost = T.size(weight='weight')
        if total_cost < min_weight:
            result_mst = T
            min_weight = total_cost
    return (result_mst, min_weight)

In [68]:
G = nx.Graph()
G.add_node(0)
G.add_node(1)
G.add_node(2)
G.add_weighted_edges_from([(0, 1, 0.5), (0, 2, 0.2)])

phase0_solve_kmst(G, 2)

(<networkx.classes.graph.Graph at 0x106423250>, 0.2)

## *Purchase Power Plants* Phase

In [65]:
def argmin(a):
    return min(range(len(a)), key=lambda x : a[x])
def argmax(a):
    return max(range(len(a)), key=lambda x : a[x])

def max_pow(P, f, ResourceCost, K):
    dp = []
    Pp = []
    for i in range(len(P) + 1):
        dp.append([0 for j in range(f + 1)])
        Pp.append([set() for j in range(f + 1)])
    for i in range(1, len(P) + 1):
        c, rt, ra, e = P[i - 1]
        running_cost = ra * ResourceCost[rt]
        for j in range(1, f + 1):
            no_change = dp[i - 1][j]
            add_plant = dp[i - 1][j - running_cost] + e
            if j - running_cost <= 0 or no_change > add_plant:
                dp[i][j] = dp[i - 1][j]
                Pp[i][j] = Pp[i - 1][j].copy()
            else:
                dp[i][j] = add_plant
                Pp[i][j] = Pp[i - 1][j - running_cost] | set(P[i - 1:i])
    if max(dp[len(P)]) < K:
        return (
            max(dp[len(P)]),
            argmax(dp[len(P)]),
            dp,
            Pp
        )
    return (
        min([ x for x in dp[len(P)] if x >= K ]),
        argmin([ x for x in dp[len(P)] if x >= K ]),
        dp,
        Pp
    )

def phase1_purchase_power_plants(P, f, P_player, P_max, ResourceCost, K):
    # No purchase decision
    m0, c0, _, _ = max_pow(P_player, f, ResourceCost, K)

    # Purchase decision, no eviction decision
    m1 = 0
    c1 = 0
    p1 = None
    for i, p in enumerate(P):
        c_i, rt, ra, e = p
        m, c, _, _ = max_pow(P_player + [p], f - c_i, ResourceCost, K) 
        c += c_i
        if m > m1 or (m == m1 and c < c1):
            m1 = m
            c1 = c
            p1 = p

    # Purchase decision, eviction decision
    m2 = 0
    c2 = 0
    p2 = None
    d2 = None
    for i, p in enumerate(P):
        c_i, rt, ra, e = p
        for pp in itertools.combinations(P_player, len(P_player) - 1):
            m, c, _, _ = max_pow([*pp, p], f - c_i, ResourceCost, K)
            c += c_i
            if m > m2 or (m == m2 and c < c2):
                m2 = m
                c2 = c
                p2 = p
                e2 = list(set(P_player) - set(pp))[0]

    out_p = None
    out_d = None
    if m0 >= m1 and m0 >= m2: # Doing nothing is best
        return out_p, out_d
    
    if m1 > m2 or (m1 == m2 and c1 <= c2):
        out_p = p1
    elif m2 > m1 or (m1 == m2 and c2 < c1):
        out_p = p2
        out_d = d2
    return out_p, out_d

In [66]:
P = [
    (10, 'Coal', 3, 4),
    (5, 'Trash', 2, 10)
]
f = 50
P_player = [
    (5, 'Uranium', 1, 1)
]
P_max = 3
ResourceCost = {
    'Coal': 1,
    'Uranium': 1,
    'Trash': 1
}
K = 10

phase1_purchase_power_plants(P, f, P_player, P_max, ResourceCost, K)

((5, 'Trash', 2, 10), None)

## *Purchasing Resources*/*Building Connections*/*Powering Plants* Phase

In [79]:
def phase2_purchase_build_power(P, Gp, C, f, ResourceCost, K, B):
    Vp = list(Gp.nodes)
    mst = list(tree.minimum_spanning_edges(G, algorithm='prim'))

    Prefix = [0]
    for i in range(1, len(mst) + 1):
        Prefix.append(Prefix[i - 1] + mst[i - 1][2]['weight'])

    MP, MC, dp, Pp = max_pow(P, f, ResourceCost, K)

    i = argmax([
        min(dp[len(P)][f - Prefix[i]], len(C) + i)
            for i in range(1, len(Vp) - len(C) + 1)
    ])
    M = len(C) + i

    if M >= K:
        i_star = min(dp[len(P)][f - Prefix[i]], len(C) + i)
    else:
        j = 0
        i_star = argmax([
            max([f - Prefix[i] - j + B * min(dp[len(P)][j], len(C) + i)])
                for j in range(0, f - Prefix[i] + 1)
        ])
        
    e = mst[0:(i_star + 1)]
    P_new = list(Pp[len(P)][f] - set(P))
    Rt = [ rt for (_, rt, _, _) in P_new ]
    Q = [ ra for (_, _, ra, _) in P_new ]
    return (Rt, Q), e, P_new

In [80]:
P_player = [
    (5, 'Uranium', 1, 1)
]
G = nx.Graph()
G.add_node(0)
G.add_node(1)
G.add_node(2)
G.add_weighted_edges_from([(0, 1, 5), (0, 2, 2)])
C = [0, 1]
f = 50
ResourceCost = {
    'Coal': 1,
    'Uranium': 1,
    'Trash': 1
}
K = 10
B = 1

phase2_purchase_build_power(P_player, G, C, f, ResourceCost, K, B)

(([], []), [(0, 2, {'weight': 2})], [])